### Note
- Below we install specific dependencies in order to train TuriCreate's object detection model.
- We install CUDA 9.1 and several python packages.
- This usually takes between 5 - 7 minutes

In [0]:
# we use /dev/null to suppress much of the output and 2>&1 so suppress errors
!rm -rf /usr/local/cuda* >/dev/null
!sudo apt-get remove --auto-remove nvidia-cuda-toolkit >/dev/null
!sudo apt-get purge nvidia-cuda-toolkit >/dev/null

# fetch cuda 9.1 from nvidia and install it
!wget https://developer.nvidia.com/compute/cuda/9.1/Prod/local_installers/cuda-repo-ubuntu1604-9-1-local_9.1.85-1_amd64 >/dev/null
!sudo dpkg -i cuda-repo-ubuntu1604-9-1-local_9.1.85-1_amd64 >/dev/null 2>&1
!sudo apt-key add /var/cuda-repo-9-1-local/7fa2af80.pub 
!sudo apt-get update >/dev/null 2>&1
!sudo apt-get install cuda-9.1 >/dev/null 2>&1

!nvcc --version

In [0]:
!pip install turicreate==5.2 >/dev/null
!pip uninstall -y mxnet 
!pip install mxnet-cu91==1.1.0
!sleep 2
!pip install numpy --upgrade >/dev/null

In [0]:
# Import libraries
import tarfile
import urllib
import turicreate as tc
tc.config.set_num_gpus(1)

In [0]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/ig02.sframe.tar.gz"
data_path = "ig02.sframe.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [0]:
## Load the data
data =  tc.SFrame('ig02.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

In [0]:
train_data.head()

In [0]:
# Create an object detection model
model = tc.object_detector.create(
    dataset=train_data,
    max_iterations=10 # to improve model accuracy, remove the max_iterations argument and TuriCreate will tune to an appropriate number
)

# Object Detector Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.object_detector.create.html#turicreate.object_detector.create

In [0]:
# Specify the CoreML model name
model_name = 'ObjectDetection'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name)